In [22]:
from sklearn.datasets import load_digits
import numpy as np

In [5]:
digits = load_digits()

In [14]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

print(digits.data.shape)
#Standardize image values with MinMaxScaler
X, y=digits.data, digits.target
scaler = MinMaxScaler()
X = scaler.fit_transform(X)

(1797, 64)


In [30]:
#Split dataset into training and test sets

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=231)

#Convert to tensors with corresponding (floats/longs to avoid errors)
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.long)
#Put tensors into tensordatasets so they are compatible with dataloader
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

# Create DataLoader objects for both training and testing sets
train_loader = DataLoader(train_dataset, batch_size=12, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=12, shuffle=False)



In [36]:
##Initialize the neural network with subclass and forward pass function
##with two actiation functions
class SimpleMLP(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(SimpleMLP, self).__init__() #call parent class with super call to initialize __init__
        self.hidden = nn.Linear(input_size, hidden_size) ##weights and biases created using nn.Linear function
        self.output = nn.Linear(hidden_size, output_size)
    
    def forward(self, x):
        x = torch.relu(self.hidden(x))
        x = torch.softmax(self.output(x), dim=1)
        return x

# Define model parameters
input_size = 64  # Each image is 8x8 pixels
hidden_size = 128
output_size = 10  # There are 10 classes (digits 0-9)

# Instantiate the model and throw it into a variable
model = SimpleMLP(input_size, hidden_size, output_size)


In [37]:
##Here we define our loss function and our optimizer or adapative learning rate
lossFunction = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [38]:
#set number of epochs (times iteration over entire dataset)

num_epochs = 15
#Train the model ins
for epoch in range(num_epochs):
    model.train() #Set model to training mode
    running_loss = 0.0
    correct = 0
    total = 0
    ##Nested loop for each epoch run the neural network 
    for inputs, labels in train_loader:
        ##Do this to stop the gradient from accumulating by default
        ##Reset Gradient to zero every back propagation 
        optimizer.zero_grad()
        outputs = model(inputs) # forward pass Compute predictions with out model
        loss = lossFunction(outputs, labels) ## backward pass compute gradients
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    
    epoch_loss = running_loss / len(train_loader)
    epoch_accuracy = (correct / total) * 100  # Convert accuracy to percentage
    
    print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {epoch_loss:.4f}, Accuracy: {epoch_accuracy:.2f}%')

Epoch [1/15], Loss: 2.1878, Accuracy: 45.72%
Epoch [2/15], Loss: 1.8054, Accuracy: 79.19%
Epoch [3/15], Loss: 1.6774, Accuracy: 85.04%
Epoch [4/15], Loss: 1.6139, Accuracy: 91.51%
Epoch [5/15], Loss: 1.5747, Accuracy: 94.02%
Epoch [6/15], Loss: 1.5530, Accuracy: 94.92%
Epoch [7/15], Loss: 1.5392, Accuracy: 95.96%
Epoch [8/15], Loss: 1.5294, Accuracy: 96.73%
Epoch [9/15], Loss: 1.5217, Accuracy: 96.80%
Epoch [10/15], Loss: 1.5153, Accuracy: 97.15%
Epoch [11/15], Loss: 1.5098, Accuracy: 97.49%
Epoch [12/15], Loss: 1.5052, Accuracy: 97.91%
Epoch [13/15], Loss: 1.5011, Accuracy: 98.19%
Epoch [14/15], Loss: 1.4977, Accuracy: 98.54%
Epoch [15/15], Loss: 1.4949, Accuracy: 98.61%


In [39]:
# evaluate the model on the test set
model.eval() #Set model to evaluation mode
with torch.no_grad(): #stop gradient computation
    correct = 0
    total = 0
    for inputs, labels in test_loader:
        outputs = model(inputs) #calculate predictions
        _, predicted = torch.max(outputs.data, 1) #calculate predicted labels
        total += labels.size(0) # increment total and correct count
        correct += (predicted == labels).sum().item()
    
    test_accuracy = correct / total
    print(f'Test Accuracy: {test_accuracy:.4f}')

# Show five example predictions along with their actual labels
example_indices = np.random.choice(len(X_test_tensor), 5, replace=False)
print("Example Predictions:")
for i in example_indices:
    output = model(X_test_tensor[i].unsqueeze(0))
    _, predicted = torch.max(output.data, 1)
    print(f'Predicted: {predicted.item()}, Actual: {y_test_tensor[i].item()}')

Test Accuracy: 0.9583
Example Predictions:
Predicted: 0, Actual: 0
Predicted: 9, Actual: 9
Predicted: 0, Actual: 0
Predicted: 7, Actual: 7
Predicted: 4, Actual: 4
